In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Veriyi yükle (CSV dosyası varsa)
df = pd.read_csv('numericData.csv')



# Veriyi incele
print("Veri özeti:")
print(df.describe())
print("\nVeri tipi:")
print(df.info())

# Eksik veri kontrolü
print(f"\n🔍 Eksik veri kontrolü:")
missing_data = df.isnull().sum()
print(missing_data)

if missing_data.any():
    print("⚠️ Eksik veriler bulundu! Ortalama ile doldurulacak...")
    df = df.fillna(df.mean())
    print("✅ Eksik veriler dolduruldu")
else:
    print("✅ Eksik veri yok!")

# Sütun isimlerini otomatik algıla ve uyarla
columns = df.columns.tolist()
print(f"\n🔧 Mevcut sütunlar: {columns}")

# Sütunları tahmin et ve eşleştir
feature_cols = []
target_col = None

for col in columns:
    col_lower = col.lower()
    if 'area' in col_lower or 'alan' in col_lower:
        feature_cols.append(col)
        print(f"📐 Alan sütunu bulundu: {col}")
    elif 'age' in col_lower or 'yas' in col_lower or 'yaş' in col_lower:
        feature_cols.append(col)
        print(f"📅 Yaş sütunu bulundu: {col}")
    elif 'floor' in col_lower or 'kat' in col_lower:
        feature_cols.append(col)
        print(f"🏢 Kat sütunu bulundu: {col}")
    elif 'price' in col_lower or 'fiyat' in col_lower:
        target_col = col
        print(f"💰 Fiyat sütunu bulundu: {col}")

# Eğer otomatik algılanamazsa, ilk 3 sütunu özellik, son sütunu hedef olarak al
if len(feature_cols) < 3 or target_col is None:
    print("⚠️ Sütunlar otomatik algılanamadı, varsayılan olarak ayarlanıyor...")
    feature_cols = columns[:3]
    target_col = columns[-1]
    print(f"📊 Özellik sütunları: {feature_cols}")
    print(f"🎯 Hedef sütun: {target_col}")

# Özellikler (X) ve hedef (y) ayır
X = df[feature_cols].values
y = df[target_col].values

print(f"\n📊 Veri boyutları:")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# Veriyi eğitim ve test olarak böl
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Veriyi normalize et (önemli!)
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

# V5 TENSORFLOW MODEL - YENİ BAŞLAYANLAR İÇİN OPTİMİZE EDİLMİŞ
print("\n🤖 TensorFlow Model V5 Oluşturuluyor...")

model = tf.keras.Sequential([
    # Giriş katmanı
    tf.keras.layers.Dense(64, activation='relu', input_shape=(len(feature_cols),), name='input_layer'),
    tf.keras.layers.BatchNormalization(),
    
    # Gizli katmanlar
    tf.keras.layers.Dense(32, activation='relu', name='hidden_1'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(16, activation='relu', name='hidden_2'),
    
    # Çıkış katmanı
    tf.keras.layers.Dense(1, name='output_layer')
])

# Model özetini göster
print("\n📋 Model Mimarisi:")
model.summary()

# V5 Model Derleme - Gelişmiş Optimizer
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='huber',  # Huber loss (outlier'lara karşı daha dayanıklı)
    metrics=['mae', 'mse']
)

# Callbacks (V5 özelliği)
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-7,
        verbose=1
    )
]

print("✅ Model derlendi ve callbacks hazırlandı")

# V5 Model Eğitimi - Gelişmiş Parametreler
print("\n🚀 Model eğitimi başlıyor...")
print(f"📊 Eğitim veri boyutu: {X_train_scaled.shape}")
print(f"📊 Test veri boyutu: {X_test_scaled.shape}")

history = model.fit(
    X_train_scaled, y_train_scaled,
    validation_data=(X_test_scaled, y_test_scaled),
    epochs=150,
    batch_size=min(32, len(X_train_scaled)//4),  # Veri boyutuna göre batch size
    callbacks=callbacks,
    verbose=1
)

print("✅ Model eğitimi tamamlandı!")

# Eğitim sonuçlarını görselleştir
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Eğitim Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Eğitim MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()

# Test verisi üzerinde tahmin yap
y_pred_scaled = model.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

# Sonuçları değerlendir
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# V5 Gelişmiş Sonuç Analizi
print(f"\n🎯 MODEL PERFORMANSI (V5):")
print(f"{'='*50}")
print(f"📊 MAE (Ortalama Mutlak Hata): {mae:,.2f}")
print(f"📊 MSE (Ortalama Kare Hata): {mse:,.2f}")
print(f"📊 RMSE (Kök Ortalama Kare Hata): {rmse:,.2f}")
print(f"📊 R² Score (Açıklama Oranı): {r2:.4f} ({r2*100:.1f}%)")
print(f"{'='*50}")

# Performans değerlendirmesi
if r2 > 0.8:
    print("🎉 Mükemmel! Model çok iyi çalışıyor")
elif r2 > 0.6:
    print("👍 İyi! Model başarılı")
elif r2 > 0.4:
    print("⚠️ Orta! Model geliştirilebilir")
else:
    print("❌ Zayıf! Model daha fazla veri veya farklı yaklaşım gerekiyor")

# Gerçek vs tahmin grafiği
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Gerçek Fiyat')
plt.ylabel('Tahmin Edilen Fiyat')
plt.title('Gerçek vs Tahmin Edilen Fiyatlar')
plt.show()

# V5 Yeni Tahmin Fonksiyonu - Sütun İsimlerini Kullan
def predict_price_v5(*args):
    """
    V5 Gelişmiş fiyat tahmin fonksiyonu
    Argümanları feature_cols sırasına göre verin
    """
    if len(args) != len(feature_cols):
        print(f"❌ Hata! {len(feature_cols)} adet parametre gerekli:")
        for i, col in enumerate(feature_cols):
            print(f"  {i+1}. {col}")
        return None
    
    new_data = np.array([list(args)])
    new_data_scaled = scaler_X.transform(new_data)
    pred_scaled = model.predict(new_data_scaled, verbose=0)
    pred_price = scaler_y.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()[0]
    
    print(f"\n🏠 TAHMİN SONUCU:")
    print(f"{'='*40}")
    for i, (col, val) in enumerate(zip(feature_cols, args)):
        print(f"📊 {col}: {val}")
    print(f"💰 Tahmin Edilen {target_col}: {pred_price:,.0f}")
    print(f"{'='*40}")
    
    return pred_price

# Örnek tahmin - gerçek veriden bir örnek al
print(f"\n🧪 ÖRNEK TAHMİN TESTI:")
if len(df) > 0:
    sample_idx = 0
    sample_data = df.iloc[sample_idx]
    actual_price = sample_data[target_col]
    
    # Tahmin yap
    feature_values = [sample_data[col] for col in feature_cols]
    predicted = predict_price_v5(*feature_values)
    
    if predicted:
        difference = abs(actual_price - predicted)
        accuracy = (1 - difference/actual_price) * 100
        print(f"✅ Gerçek fiyat: {actual_price:,.0f}")
        print(f"📈 Tahmin doğruluğu: %{accuracy:.1f}")

# Modeli kaydet
model_filename = 'ev_fiyat_model_v5.h5'
model.save(model_filename)
print(f"\n💾 Model '{model_filename}' olarak kaydedildi!")
print("🎉 TensorFlow V5 Model hazır!")

Veri özeti:
       living_room         area          age        floor          price
count       7313.0  7313.000000  7313.000000  7313.000000    7313.000000
mean           1.0   100.619445    20.032682     2.487488   31312.125393
std            0.0    33.257573    13.694698     1.867577   18209.545802
min            1.0    10.000000     0.000000    -3.000000    5000.000000
25%            1.0    75.000000     7.000000     1.000000   19000.000000
50%            1.0    95.000000    20.000000     2.000000   25000.000000
75%            1.0   120.000000    30.000000     4.000000   38000.000000
max            1.0   230.000000    65.000000     8.000000  105000.000000

Veri tipi:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7313 entries, 0 to 7312
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   living_room  7313 non-null   int64
 1   area         7313 non-null   int64
 2   age          7313 non-null   int64
 3   floor   

c:\Users\fatih\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: All layers added to a Sequential model should have unique names. Name 'input_layer' is already the name of a layer in this model. Update the `name` argument to pass a unique name.